In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

In [2]:
print(torch.__version__)

1.5.0


In [3]:
LR = 1e-3
BATCH_SIZE = 500
EPOCHS = 1

In [4]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [5]:
x_trn = datasets.MNIST('data', train=True, transform=transform, download=True)
x_tst = datasets.MNIST('data', train=False, transform=transform)

In [6]:
trn_loader = torch.utils.data.DataLoader(x_trn, batch_size=BATCH_SIZE)
tst_loader = torch.utils.data.DataLoader(x_tst, batch_size=BATCH_SIZE)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [8]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [9]:
log_interval = 40
model.train()

for epoch in range(1, EPOCHS + 1):
    for idx, (data, target) in enumerate(trn_loader):
        data, target = data, target
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{idx * len(data)}/{len(trn_loader.dataset)}"
                  f" ({100. * idx / len(trn_loader):.0f}%)]\tLoss: {loss.item():.6f}")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307046
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.250224
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.136825


In [10]:
tst_loss = 0
correct = 0
model.eval()

with torch.no_grad():
    for data, target in tst_loader:
        data, target = data, target
        output = model(data)
        tst_loss += F.nll_loss(output, target, reduction='sum').item() 
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
tst_loss /= len(tst_loader.dataset)

print(f"\nTest set: Average loss: {tst_loss:.4f},"
      f" Accuracy: {correct}/{len(tst_loader.dataset)}"
      f" ({100. * correct / len(tst_loader.dataset):.0f}%)\n")


Test set: Average loss: 0.0642, Accuracy: 9783/10000 (98%)



In [11]:
torch.save(model.state_dict(), "model.pt")

In [14]:
!kubectl apply -f torch.yaml

inferenceservice.serving.kubeflow.org/pytorch-mnist created


In [ ]:
import json

data = {
    "instances": [
        [6.8, 2.8, 4.8, 1.4],
        [5.1, 3.5, 1.4, 0.2]
    ]
}

headers = {"content-type": "application/json", "Host": "xgboost-example.kubeflow-user.svc.cluster.local"}
response = requests.post("http://cluster-local-gateway.istio-system/v1/models/xgboost-example:predict", json=data, headers=headers)

In [18]:
next(iter(tst_loader))[0][0].shape

torch.Size([1, 28, 28])